In [1]:
import numpy as np

In [16]:
def reconstruct(groups, group_sizes, strong_set, strong_order, strong_begins, strong_beta, indices, vals):
    out = np.copy(strong_beta)

    if len(vals) == 0:
        out[...] = 0
        return out
    
    outer_pos = 0
    inner_pos = 0
    so = strong_order[outer_pos]
    group = groups[strong_set[so]]
    group_size = group_sizes[strong_set[so]]

    for idx, val in zip(indices, vals):
        while outer_pos < len(strong_set):
            begin_ = strong_begins[so] + inner_pos
            inner_pos = min(group_size, idx - group)
            out[begin_ : strong_begins[so] + inner_pos] = 0
            
            do_break = False
            if inner_pos < group_size:
                out[strong_begins[so] + inner_pos] = val
                inner_pos += 1
                do_break = True

            if inner_pos == group_size:
                inner_pos = 0
                outer_pos += 1

                if outer_pos < len(strong_set):
                    so = strong_order[outer_pos]
                    group = groups[strong_set[so]]
                    group_size = group_sizes[strong_set[so]]
                    
            if do_break: break

    if outer_pos == len(strong_set):
        return out

    begin_ = strong_begins[so] + inner_pos
    out[begin_ : strong_begins[so] + group_size] = 0
    outer_pos += 1
        
    while outer_pos < len(strong_set):
        so = strong_order[outer_pos]
        group_size = group_sizes[strong_set[so]]
        begin_ = strong_begins[so]
        out[begin_ : begin_ + group_size] = 0
        outer_pos += 1
        
    return out

In [17]:
def checker(p, indices, vals, strong_set, groups, group_sizes):
    out = np.zeros(p)
    for i, v in zip(indices, vals):
        out[i] = v
    
    strong_beta = np.concatenate([
        out[groups[i] : (groups[i] + group_sizes[i])]
        for i in strong_set
    ])
    return strong_beta, out

In [18]:
np.random.seed(0)

groups = np.array([0, 2, 5, 10, 21, 25, 30, 40, 50, 80], dtype=int)
strong_set = np.array([3, 2, 0, 5, 7, 8], dtype=int)

group_sizes = (groups[1:] - groups[:-1])
groups = groups[:-1]
p = np.sum(group_sizes, dtype=int)
strong_order = np.argsort(strong_set)
strong_begins = np.cumsum([0] + [group_sizes[i] for i in strong_set])[:-1]
strong_beta = np.concatenate(
    [
        np.random.normal(0, 1, (group_sizes[i],))
        for i in strong_set
    ]
)

indices = np.sort(np.concatenate([np.arange(groups[s], groups[s] + group_sizes[s]) for s in strong_set]))
indices_sub = np.sort(np.random.choice(indices, size=7, replace=False))
vals_sub = np.array(np.random.normal(0, 1, len(indices_sub)), dtype=int)
expected = checker(p, indices_sub, vals_sub, strong_set, groups, group_sizes)[0]

In [19]:
strong_set = [19, 39, 60, 77, 90, 37, 40, 50, 53, 70, 76, 89]
strong_begins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
strong_order = np.argsort(strong_set)
groups = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
group_sizes = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
strong_beta = [0.35652244550831164,
   -0.43441266168717885,
   -0.5331579888034386,
   0.32242026434983856,
   0.3463288813909085,
   0.0,
   0.11507676683122493,
   -0.1314169595298529,
   -0.10279881846983285,
   0.04044473116685074,
   -0.014382451125130237,
   0.06920045903500745]

indices_sub = [19, 37, 39, 40, 50, 53, 60, 70, 76, 77, 89, 90]
vals_sub = np.array([[ 0.35652245],
[-0.43441266],
[-0.53315799],
[ 0.32242026],
[ 0.34632888],
[ 0.07129225],
[ 0.11507677],
[-0.13141696],
[-0.10279882],
[ 0.04044473],
[-0.01438245],
[ 0.06920046]])[:,0]

expected = np.array([[ 0.35652245],
        [-0.43441266],
        [-0.53315799],
        [ 0.32242026],
        [ 0.34632888],
        [ 0.07129225],
        [ 0.11507677],
        [-0.13141696],
        [-0.10279882],
        [ 0.04044473],
        [-0.01438245],
        [ 0.06920046]])[np.argsort(strong_order), 0]
        
assert len(groups) == len(group_sizes)
assert len(strong_set) == len(strong_begins)
assert len(strong_set) == len(strong_order)
assert len(strong_set) == len(strong_beta)
assert len(indices_sub) == len(vals_sub)
assert len(strong_beta) == len(expected)

In [20]:
actual = reconstruct(
    groups, group_sizes, strong_set, strong_order, strong_begins, strong_beta, indices_sub, vals_sub
)
np.all(actual == expected), expected, actual

(True,
 array([ 0.35652245, -0.53315799,  0.11507677,  0.04044473,  0.06920046,
        -0.43441266,  0.32242026,  0.34632888,  0.07129225, -0.13141696,
        -0.10279882, -0.01438245]),
 array([ 0.35652245, -0.53315799,  0.11507677,  0.04044473,  0.06920046,
        -0.43441266,  0.32242026,  0.34632888,  0.07129225, -0.13141696,
        -0.10279882, -0.01438245]))